In [ ]:
class NaiveBayesGenerative:
    def fit(self, X, y):
        # Estimate P(y) - class probabilities
        self.class_probs = np.bincount(y) / len(y)
        
        # Estimate P(x|y) - feature distributions per class
        self.feature_probs = {}
        for class_label in [0, 1]:
            class_data = X[y == class_label]
            # Estimate feature distributions for this class
            self.feature_probs[class_label] = self._estimate_distributions(class_data)
    
    def predict_proba(self, X):
        # Using Bayes rule: P(y|x) ∝ P(x|y)P(y)
        return self._compute_posterior(X)

class LogisticDiscriminative:
    def fit(self, X, y):
        # Directly model P(y|x) without modeling P(x|y)
        self.weights = self._optimize_likelihood(X, y)
    
    def predict_proba(self, X):
        # Directly estimate P(y|x)
        return self._sigmoid(X @ self.weights)

In [2]:
import numpy as np
from scipy.stats import multivariate_normal

class GenerativeClassifier:
    def __init__(self):
        self.class_means = {}
        self.class_covs = {}
        self.class_priors = {}
    
    def fit(self, X, y):
        # For each class, we model P(x|y) as a Gaussian
        # and P(y) as the empirical class frequency
        unique_classes = np.unique(y)
        n_samples = len(y)
        
        for c in unique_classes:
            # Get samples for this class
            X_c = X[y == c]
            
            # Estimate P(y) - class prior
            self.class_priors[c] = len(X_c) / n_samples
            
            # Estimate P(x|y) - class-conditional density
            self.class_means[c] = np.mean(X_c, axis=0)
            self.class_covs[c] = np.cov(X_c, rowvar=False)
    
    def predict_proba(self, X):
        # Using Bayes rule: P(y|x) ∝ P(x|y)P(y)
        probs = []
        for c in self.class_priors:
            # Calculate P(x|y)
            likelihood = multivariate_normal.pdf(
                X, 
                mean=self.class_means[c], 
                cov=self.class_covs[c]
            )
            # Multiply by P(y)
            class_prob = likelihood * self.class_priors[c]
            probs.append(class_prob)
            
        # Normalize to get probabilities
        probs = np.array(probs).T
        return probs / probs.sum(axis=1, keepdims=True)

class DiscriminativeClassifier:
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate
        self.weights = None
        self.bias = None
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y, epochs=100):
        # Directly model P(y|x) through logistic regression
        n_features = X.shape[1]
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        for _ in range(epochs):
            # Forward pass
            z = X.dot(self.weights) + self.bias
            y_pred = self.sigmoid(z)
            
            # Gradient descent
            dw = X.T.dot(y_pred - y) / len(y)
            db = np.mean(y_pred - y)
            
            self.weights -= self.lr * dw
            self.bias -= self.lr * db
    
    def predict_proba(self, X):
        z = X.dot(self.weights) + self.bias
        return self.sigmoid(z)
    
def demonstrate_model_differences():
    # Generate synthetic data
    np.random.seed(42)
    n_samples = 1000
    
    # Class 0: Gaussian centered at (0,0)
    X0 = np.random.multivariate_normal([0,0], [[1,0.5],[0.5,1]], n_samples//2)
    # Class 1: Gaussian centered at (2,2)
    X1 = np.random.multivariate_normal([2,2], [[1.5,-0.5],[-0.5,1.5]], n_samples//2)
    
    X = np.vstack([X0, X1])
    y = np.array([0]*(n_samples//2) + [1]*(n_samples//2))
    
    # Train both models
    gen_model = GenerativeClassifier()
    disc_model = DiscriminativeClassifier()
    
    gen_model.fit(X, y)
    disc_model.fit(X, y)
    
    # Now we can show a key difference:
    # Generative model can generate new samples
    def generate_samples(gen_model, n_samples=10):
        samples = []
        for _ in range(n_samples):
            # First sample a class according to P(y)
            class_idx = np.random.choice(
                list(gen_model.class_priors.keys()),
                p=list(gen_model.class_priors.values())
            )
            # Then sample from P(x|y) for that class
            sample = np.random.multivariate_normal(
                gen_model.class_means[class_idx],
                gen_model.class_covs[class_idx]
            )
            samples.append(sample)
        return np.array(samples)
    
    # Generate new samples
    new_samples = generate_samples(gen_model)
    # Discriminative model cannot generate samples!
    
    return X, y, new_samples

demonstrate_model_differences()

(array([[-0.36103492, -0.49929923],
        [-1.32242966,  0.2006002 ],
        [ 0.31985125,  0.08571429],
        ...,
        [ 2.40526228,  3.1081587 ],
        [ 1.40294171,  1.34989919],
        [ 1.63634125,  1.31020732]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 

In [3]:
def likelihood_ratio_test(complex_model, simple_model, data):
    # Compute log-likelihoods
    ll_complex = complex_model.log_likelihood(data)
    ll_simple = simple_model.log_likelihood(data)
    
    # Compute test statistic
    lr_statistic = 2 * (ll_complex - ll_simple)
    
    # Degrees of freedom = difference in number of parameters
    df = complex_model.n_params - simple_model.n_params
    
    # p-value from chi-squared distribution
    from scipy.stats import chi2
    p_value = 1 - chi2.cdf(lr_statistic, df)
    
    return lr_statistic, p_value